In [1]:
import numpy as np
import pImpactR as impact
import matplotlib.pyplot as plt
int = np.vectorize(int)

In [ ]:
nTurn = 1024

In [ ]:
def resonnace(ix,iy,N):
    if ix==0 and iy==0:
        nux=0
        nuy=0
    else:
        if ix==0:
            nux = np.linspace(-1,1,2)
            nuy = np.linspace(float( N )/iy, float( N )/iy, 2)
        elif iy==0:
            nuy = np.linspace(-1,1,2)
            nux = np.linspace(float( N )/ix, float( N )/ix, 2)
        else:        
            nux = np.linspace(-1,1,2)
            nuy = ix/float(iy)*nux +  N/float(iy)
    return nux,nuy


plt.figure(figsize=(4,4))
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.4,ls='--')

plt.xlim((0,1))
plt.ylim((0,1))

In [ ]:
beam, lattice = impact.readInputFile('test.SF.in')
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

betx = lattice[2].betx
alfx = 0.0
NL_c = lattice[2].transverse_scale_c
x0 = betx**0.5*NL_c
x0 = [-x0,x0]
y0 = [0,0]

In [ ]:
iTBT,TBT = impact.readTBT(2048,ke,mass,freq)
TBT=TBT[:,:4,:]
Y=TBT[:,2,:] - 1j*TBT[:,3,:]
TBT=TBT[:,:2,:]
X=TBT[:,0,:] - 1j*TBT[:,1,:]
TBT=None
npt = len(iTBT)

In [ ]:
tuneXbefore= np.zeros([1,npt])
tuneXafter = np.zeros([3,npt])
tuneYbefore= np.zeros([1,npt])
tuneYafter = np.zeros([3,npt])
for i in range(nptLive):
    tuneXbefore[:,i],amp,dummy = impact.util.naff(1,X[:nTurn,i],window_id=1)
    tuneXafter [:,i],amp,dummy = impact.util.naff(3,X[nTurn:,i],window_id=1)
    tuneYbefore[:,i],amp,dummy = impact.util.naff(1,Y[:nTurn,i],window_id=1)
    tuneYafter [:,i],amp,dummy = impact.util.naff(3,Y[nTurn:,i],window_id=1)

In [ ]:
def getClosestDiff(ref,arr):
    # ref.shape = nSample
    nMode = len(arr)
    if ref>1.0:
        ref = ref - 1.0
    for i in range(nMode):
        if arr[i]>1.0:
            arr[i]=arr[i]-1.0
        if arr[i]<=0.0:
            arr[i]=arr[i]+1.0
    diff = np.abs(ref - arr)
    for i in range(nMode):
        if (diff[i]>0.5):
            diff[i] = 1.0 - diff[i]
    return np.min(diff)

In [ ]:
Xdiff = getClosestDiff(tuneXbefore,tuneXafter)
Ydiff = getClosestDiff(tuneYbefore,tuneYafter)
Adiff = np.sqrt(Xdiff**2 +Ydiff**2)

AdiffLog = np.log10(Adiff+1.0e-6)
XdiffLog = np.log10(Xdiff+1.0e-6)
YdiffLog = np.log10(Ydiff+1.0e-6)

iMaxX = np.argmax(Xdiff)
iMaxY = np.argmax(Ydiff)
iMaxA = np.argmax(Adiff)
iMinX = np.argmin(Xdiff)
iMinY = np.argmin(Ydiff)
iMinA = np.argmin(Adiff)

# plot

In [ ]:
xmax = np.max(np.real(X[0,:]))
ymax = np.max(np.real(Y[0,:]))
nx = 50
ny = int(nx*ymax/xmax)
dx = xmax/nx
dy = ymax/ny
Grid = impact.data.dictClass
Grid.x = np.arange(nx)*dx + 0.5*dx
Grid.y = np.arange(ny)*dy + 0.5*dy
Grid.c = np.zeros([nx,ny])

In [5]:
nSum = np.zeros([nx,ny],dtype=np.int)
for ipt in range(npt)
    for ix in range(nx):
        if ix*dx < np.real(X[0,ipt]) <= (ix+1)*dx:
            for iy in range(ny):
                if iy*dy < np.real(Y[0,ipt]) <= (iy+1)*dy:
                    nSum[ix,iy]   = nSum[ix,iy]+1
                    Grid.c[ix,iy] = Grid.c[ix,iy] + AdiffLog[ipt]

for ix in range(nx):
    for iy in range(ny):
        if nSum[ix,iy]>0:
            Grid.c[ix,iy] = Grid.c[ix,iy]/nSum[ix,iy]

### FFT

In [ ]:
n=2048
plt.figure(figsize=[8,6])
plt.subplot(3,2,1)
plt.semilogy(np.abs(np.fft.fft(X[n][:,iMax[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][:,iMin[n][1]])),'k')
plt.xlabel('fft$_x$')
plt.subplot(3,2,2)
plt.semilogy(np.abs(np.fft.fft(Y[n][:,iMax[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][:,iMin[n][2]])),'k')
plt.legend(['most chaotic','most regular'],loc=1)
plt.xlabel('fft$_y$')
plt.subplot(3,2,3)
plt.semilogy(np.abs(np.fft.fft(X[n][:1024,iMin[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][1024:,iMin[n][1]])),'k')
plt.xlabel('fft_regular_comparison$_x$')
plt.subplot(3,2,4)
plt.semilogy(np.abs(np.fft.fft(Y[n][:1024,iMin[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][1024:,iMin[n][2]])),'k')
plt.xlabel('fft_regular_comparison$_y$')
plt.legend(['1st half','2nd half'],loc=1)
plt.subplot(3,2,5)
plt.semilogy(np.abs(np.fft.fft(X[n][:1024,iMax[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][1024:,iMax[n][1]])),'k')
plt.xlabel('fft_chotic_comparison$_x$')
plt.subplot(3,2,6)
plt.semilogy(np.abs(np.fft.fft(Y[n][:1024,iMax[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][1024:,iMax[n][2]])),'k')
plt.xlabel('fft_chotic_comparison$_y$')
plt.legend(['1st half','2nd half'],loc=1)
plt.tight_layout()
plt.savefig('FMA.FFT.png',dpi=180)
# plt.close()

##### fft of p at center

In [ ]:
iptCore=np.argmin(np.abs(X[0,:])*np.abs(X[0,:])+np.abs(Y[0,:])*np.abs(Y[0,:]))

plt.figure(figsize=[4,6])
plt.subplot(3,1,1)
plt.semilogy(np.abs(np.fft.fft(X[:,iptCore])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[:,iptCore])),'k')
plt.xlabel('fft_core')
plt.subplot(3,1,2)
plt.semilogy(np.abs(np.fft.fft(X[:1024,iptCore])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[1024:,iptCore])),'k')
plt.xlabel('fft_core_comparison$_x$')
plt.legend(['1st half','2nd half'],loc=1)
plt.subplot(3,1,3)
plt.semilogy(np.abs(np.fft.fft(Y[:1024,iptCore])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[1024:,iptCore])),'k')
plt.xlabel('fft_core_comparison$_y$')
plt.legend(['1st half','2nd half'],loc=1)
plt.tight_layout()
# plt.close()

print(Adiff[iMin[0]],Adiff[iptCore],Adiff[iMax[0]])

### tune plot

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.4,ls='--')
f1=plt.scatter(tuneXbefore,tuneYbefore,c=AdiffLog,s=1.0,marker='s')
   
plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)

plt.xlim(0.1,0.26)
plt.ylim(0.01,0.2)
plt.tight_layout()
plt.savefig('FMA.tune.A.png',dpi=180)
# plt.close()

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.4,ls='--')
f1=plt.scatter(tuneXbefore,tuneYbefore,c=XdiffLog,s=1.0,marker='s')
plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(0.1,0.26)
plt.ylim(0.01,0.2)
plt.tight_layout()
plt.savefig('FMA.tune.X.png',dpi=180)
# plt.close()

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.4,ls='--')
f1=plt.scatter(tuneXbefore,tuneYbefore,c=YdiffLog,s=1.0,marker='s')
plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(0.1,0.26)
plt.ylim(0.01,0.2)
plt.tight_layout()
plt.savefig('FMA.tune.Y.png',dpi=180)
# plt.close()

### Poincare

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[:,iMax[0]]),np.imag(X[:,iMax[0]]),c='r',s=0.1)
plt.scatter(np.real(X[:,iMin[0]]),np.imag(X[:,iMin[0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('px')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.Poincare.A.png',dpi=180)

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[:,iMax[0]]),np.real(Y[:,iMax[0]]),c='r',s=0.1)
plt.scatter(np.real(X[:,iMin[0]]),np.real(Y[:,iMin[0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.Poincare.X.png',dpi=180)

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(Y[:,iMax[0]]),np.imag(Y[:,iMax[0]]),c='r',s=0.1)
plt.scatter(np.real(Y[:,iMin[0]]),np.imag(Y[:,iMin[0]]),c='k',s=0.1)
plt.xlabel('y')
plt.ylabel('py')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.Poincare.Y.png',dpi=180)